In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from sqlalchemy import create_engine

import os
import csv
import re

from datetime import datetime

In [2]:
from sklearn.externals import joblib ## model persistance

In [3]:
pkloc = '/home/master/Data/dummy/'
reploc = pkloc
loc = ''
db_name = 'random_train.db'

###  # create report.csv file with header (Time_taken) * rest adds auto

In [4]:
def save_report(msg,di):               # create csv file with header
    rep = "\n"+msg+" :\n\t"+str(di)
    fp = open(reploc+'report.txt','a')
    fp.write(rep)
    fp.close()

def save_report_csv(msg,di):               # create csv file with header
    cn = pd.read_csv(reploc+'report.csv')
    for k,v in di.items():
        cn.loc[msg,k] = v
    #cn.to_csv(reploc+'report.csv')
    return cn
    

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(loc+db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
 
    return None

In [5]:
con = create_connection(db_name)

In [6]:
tag_rm = list( pd.read_csv('tag_to_remove.csv')['Tags'] )
t = [str(i) for i in tag_rm]
li_string = "('"+ "','".join(t) +"')" # li_string = '("'+ '","'.join(t) +'")' #  ' vs "

In [7]:
if con is not None:
    tag_data = pd.read_sql('SELECT Tags FROM data WHERE Tags NOT IN '+li_string,con)
else :
    print('Conn error')

In [8]:
print(tag_data.head())
print('Shape',tag_data.shape)

                                                Tags
0             iphone objective-c ios uiview uibutton
1                    svg internet-explorer-9 raphael
2  validation spring-mvc internationalization cus...
3                             windows java copy text
4                                  javascript jquery
Shape (9708, 1)


In [9]:
if con is not None:
    t_data = pd.read_sql('SELECT Title FROM data WHERE Tags NOT IN '+li_string,con)
else :
    print('Conn error')


In [10]:
print(t_data.head())
print('Shape',t_data.shape)

                                               Title
0       One tap triggering events on multiple views?
1  IE9 text positioning bug when zoomed in with R...
2  Spring MVC custom errors and internationalization
3                How to copy text from Java program?
4  How to scroll to a part of the page using jQuery?
Shape (9708, 1)


## Preprocess title Data

In [11]:
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")
# https://stackoverflow.com/questions/35345761/python-re-split-vs-nltk-word-tokenize-and-sent-tokenize

In [12]:

#t_data.head()
#t_data.Title = t_data.Title.apply(lambda x : x.encode('utf-8'))
t_data.Title = t_data.Title.apply(lambda x : str.lower(str(x)))
t_data.Title = t_data.Title.apply(lambda x : re.sub(r'[^A-Za-z0-9#+.\-]+',' ',x))
#title_data = t_data


In [13]:
def fn(sen):
    return ' '.join(str(stemmer.stem(w)) for w in word_tokenize(sen) if w not in stop_words and (len(w)!=1 or w=='c'))

#' '.join for w in word_tokenize(x) if w not in stop_words)


In [14]:
t_data.Title = t_data.Title.apply(lambda x : fn(x))

In [15]:
t_data.Title[:5]

0     one tap trigger event multipl view
1    ie9 text posit bug zoom raphael svg
2    spring mvc custom error internation
3                 copi text java program
4            scroll part page use jqueri
Name: Title, dtype: object

In [16]:
title_data = t_data

### ----- Body

In [ ]:
if con is not None:
    b_data = pd.read_sql('SELECT Body FROM data  WHERE Tags NOT IN '+li_string,con)
else :
    print('Conn error')

In [ ]:
print(b_data[:5])
print('Shape',b_data.shape)

In [ ]:
b_data.Body = b_data.Body.apply(lambda x : x.encode('utf-8'))
b_data.Body = b_data.Body.apply(lambda x : str.lower(str(x)))
b_data.Body = b_data.Body.apply(lambda x : re.sub(r'<code>(.*?)</code>',' ',x))
b_data.Body = b_data.Body.apply(lambda x : re.sub(r'<.*?>',' ',x))
b_data.Body = b_data.Body.apply(lambda x : re.sub(r'[^A-Za-z0-9#+.\-]+',' ',x))

b_data.Body = b_data.Body.apply(lambda x : fn(x))

In [ ]:
b_data[:5]

In [ ]:
title_data = b_data

## ---------------- title+body = ques

In [ ]:
b_data.shape

In [ ]:
t_data.shape

In [ ]:
title_data = pd.DataFrame()
title_data['ques'] = t_data.Title + b_data.Body
title_data.shape

### --------------------------------------

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
vectorizer_tag = CountVectorizer(tokenizer = lambda x: x.split(), binary='true',min_df = 5)  # min_df - document frequency strictly lower than the given threshold

In [19]:
multi_lab_y = vectorizer_tag.fit_transform(tag_data.Tags)

In [20]:
vectorizer_tag

CountVectorizer(analyzer='word', binary='true', decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function <lambda> at 0x7fb8a68a7620>, vocabulary=None)

In [21]:
# joblib.dump(tag_data.Tags,pkloc+'tag_data.pkl')

In [22]:
print("Number of data points :", multi_lab_y.shape[0])
print("Number of unique tags :", multi_lab_y.shape[1])

Number of data points : 9708
Number of unique tags : 939


In [ ]:
#vectorizer_tag.get_feature_names()

In [ ]:
## choose first n tags (desc order of count)

def tags_to_choose(n):
    t = multilabel_y.sum(axis=0).tolist()[0]
    sorted_tags_i = sorted(range(len(t)), key=lambda i: t[i], reverse=True)
    multilabel_yn = multilabel_y[:,sorted_tags_i[:n]]
    return multilabel_yn

<h2>4.2 Split the data into test and train (80:20) </h2>

In [23]:
tot_size = title_data.shape[0]
train_size = int(tot_size * 0.8)
print(tot_size,train_size)

9708 7766


In [24]:
x_train=title_data.head(train_size)
x_test=title_data.tail(tot_size - train_size)

y_train = multi_lab_y[0:train_size,:]
y_test = multi_lab_y[train_size:tot_size,:]

In [ ]:
# joblib.dump(y_test,pkloc+'y_test.pkl')

## @ Model Testing

In [25]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import LinearSVC

from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score

In [26]:
vectorizer = TfidfVectorizer(min_df=0.00009, max_features=200000, smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,3))

In [27]:
start = datetime.now()
x_train_multilabel = vectorizer.fit_transform(x_train.Title)     ## replace column name appropiately
x_test_multilabel = vectorizer.transform(x_test.Title)
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:00.396779


In [28]:
print("Dimensions of train data X:",x_train_multilabel.shape, "Y :",y_train.shape)
print("Dimensions of test data X:",x_test_multilabel.shape,"Y:",y_test.shape)

Dimensions of train data X: (7766, 64691) Y : (7766, 939)
Dimensions of test data X: (1942, 64691) Y: (1942, 939)


In [ ]:
joblib.dump(x_test_multilabel,pkloc+'body_xtest_multilabel.pkl')

### SGDC with OnevsRest

In [ ]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='hinge', alpha=0.00001, penalty='l1'), n_jobs=-1)
classifier.fit(x_train_multilabel, y_train)
predictions = classifier.predict(x_test_multilabel)

time_t = datetime.now() - start
#print("macro f1 score :",metrics.f1_score(y_test, predictions, average = 'macro'))
#print("micro f1 scoore :",metrics.f1_score(y_test, predictions, average = 'micro'))
#print("hamming loss :",metrics.hamming_loss(y_test,predictions))
#print("Precision recall report :\n",metrics.classification_report(y_test, predictions))


In [ ]:
rep_di = dict()
rep_di["Time_taken"] = str(time_t)
rep_di["accuracy"] = metrics.accuracy_score(y_test,predictions)
rep_di["micro_f1_scoore"] = metrics.f1_score(y_test, predictions, average = 'micro')
print(str(rep_di))

In [ ]:
save_report("SGDC with OnevsRest - hinge",rep_di)

In [ ]:
joblib.dump(classifier,pkloc+'title_sgdc_hinge.pkl')

### Logistic with OnevsRest

In [ ]:
start = datetime.now()

classifier2 = OneVsRestClassifier(LogisticRegression(penalty='l1'), n_jobs=-1)
classifier2.fit(x_train_multilabel, y_train)
predictions2 = classifier2.predict(x_test_multilabel)

time_t = datetime.now() - start

In [ ]:

rep_di = dict()
rep_di["Time_taken"] = str(time_t)
rep_di["accuracy"] = metrics.accuracy_score(y_test,predictions2)
rep_di["micro_f1_scoore"] = metrics.f1_score(y_test, predictions2, average = 'micro')
print(str(rep_di))

In [ ]:
save_report("Logistic with OnevsRest",rep_di)

In [ ]:
joblib.dump(classifier2,pkloc+'body_logistic.pkl')

## LDA  -- TIME

In [ ]:
y_train3 =y_train.todense()
y_test3 = y_test.todense()

print type(y_train)
print type(x_train_multilabel.todense())

In [ ]:
start = datetime.now()

#classifier3 = OneVsRestClassifier(LDA())
#classifier3.fit(x_train_multilabel.todense(), y_train)
#predictions3 = classifier3.predict(x_test_multilabel)

#print("Time taken to run this cell :", datetime.now() - start)
#print("accuracy :",metrics.accuracy_score(y_test,predictions3))

## SVC

In [ ]:
start = datetime.now()

classifier4 = OneVsRestClassifier(LinearSVC(), n_jobs=-1)
classifier4.fit(x_train_multilabel, y_train)
predictions4 = classifier4.predict(x_test_multilabel)

time_t = datetime.now() - start


In [ ]:

rep_di = dict()
rep_di["Time_taken"] = str(time_t)
rep_di["accuracy"] = metrics.accuracy_score(y_test,predictions4)
rep_di["micro_f1_scoore"] = metrics.f1_score(y_test, predictions4, average = 'micro')
print(str(rep_di))

In [ ]:
save_report("SVC",rep_di)

In [ ]:
joblib.dump(classifier4,pkloc+'body_svc.pkl')

## MLkNN

In [29]:
from skmultilearn.adapt import MLkNN
classifier_k = MLkNN(k=101)

In [ ]:
classifier_k.fit(x_train_multilabel, y_train)

In [ ]:
predictions5 = classifier_k.predict(x_test_multilabel)

In [ ]:
rep_di = dict()
rep_di["Time_taken"] = str(time_t)
rep_di["accuracy"] = metrics.accuracy_score(y_test,predictions5)
rep_di["micro_f1_scoore"] = metrics.f1_score(y_test, predictions5, average = 'micro')
print(str(rep_di))

In [ ]:
save_report("MLkNN 21",rep_di)

In [ ]:
type(y_test)

In [ ]:
actual = vectorizer_tag.inverse_transform(y_test)
actual = [' '.join(i) for i in actual]
actual = pd.Series(actual,name="Actual")
#actual

In [ ]:
preds = vectorizer_tag.inverse_transform(predictions)
preds = [' '.join(i) for i in preds]
preds = pd.Series(preds,name="Pred_by_ques")
#preds

In [ ]:
resul_df = pd.concat([actual,preds],axis=1)
#resul_df.to_csv("01_resul_rep.csv",index=None)

In [ ]:
#rr = pd.read_csv('01_resul_rep.csv')

In [ ]:
#rr.append?

In [ ]:
rr['pred_by_ques']=preds

In [ ]:
rr.to_csv("01_rep_mindf_03.csv",index=None)

In [ ]:
r = pd.read_csv('01_rep_mindf_03.csv')

In [ ]:
#r

## load and run

In [ ]:
title_x_test_multilabel = joblib.load('/data/title_xtest_multilabel.pkl')

title_sgc = joblib.load('/data/title_sgdc.pkl')

pred_title = title_sgc.predict(title_x_test_multilabel)

print("From title accuracy :",metrics.accuracy_score(y_test,pred_title))
print("From title micro f1 scoore :",metrics.f1_score(y_test, pred_title, average = 'micro'))

In [ ]:
body_x_test_multilabel = joblib.load('/data/body_xtest_multilabel.pkl')

body_sgc = joblib.load('/data/body_sgdc.pkl')

pred_body = body_sgc.predict(body_x_test_multilabel)

print("From body accuracy :",metrics.accuracy_score(y_test,pred_body))
print("From body micro f1 scoore :",metrics.f1_score(y_test, pred_body, average = 'micro'))

In [ ]:
p_or = pred_title.todense() | pred_body.todense()

In [ ]:
print("From OR accuracy :",metrics.accuracy_score(y_test,p_or))
print("From OR micro f1 scoore :",metrics.f1_score(y_test, p_or, average = 'micro'))

In [ ]:
preds = vectorizer_tag.inverse_transform(p_or)
preds = [' '.join(i) for i in preds]
preds = pd.Series(preds,name="Pred_by_OR")
r['pred_by_OR'] = preds
r.to_csv("01_rep_mindf_03.csv",index=None)

In [ ]:
r = pd.read_csv('01_rep_mindf_03.csv')
r